# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x110fb2410>: 71, mem=True, ro=False: entities=9, map=2


In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [4]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def cutoffs(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        
        clock = valuable.getObj(_tr.RootClock, 'Main')
        
        def cuts(ts):
            def fn(node):
                obj = node.key.object()
                m = node.key.shortName()
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return nodes

        ret = {}
        for n in cuts(ts1):
            ret[n.key._key] = n
        for n in cuts(ts2):
            ret[n.key._key] = n
        return ret.values()
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        nodes = self.cutoffs()
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.object().meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n] = tsAmend
            name = n.object().meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n] = ts2
            name = n.object().meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            displayMarkdown(node.footnoteMarkdown())
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [5]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [6]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in funtion
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|50,744,643|27|50,744,643|GetValue
|Report:data|1|50,744,616|953|50,744,616|GetValue/Calc
|TradingContainer:NPV|11|50,556,085|298|4,596,007|GetValue
|TradingContainer:NPV|11|50,555,787|199|4,595,980|GetValue/Calc
|Portfolio:items|121|50,087,675|2,846|413,947|GetValue
|Portfolio:items|121|50,087,440|2,069|413,945|GetValue/Calc
|Workbook:items|1,104|47,437,208|28,316|42,968|GetValue
|Workbook:items|1,100|47,408,891|21,136|43,098|GetValue/Calc
|Report:cutoffs|1|28,582,694|72|28,582,694|GetValue
|Report:cutoffs|1|28,582,622|340|28,582,622|GetValue/Calc
|Root:Clocks|548|28,582,255|5,712,073|52,157|Context
|TradeOpenEvent:ticket|22,286|10,649,654|297,679|477|GetValue
|TradeOpenEvent|1,014|10,547,915|10,547,915|10,402|Db.Get
|TradingTicket|1,014|10,351,974|10,351,974|10,209|Db.Get
|Root:Activity MarketData|255|3,229,382|3,032,419|12,664|Context
|Root:Start breaks|255|3,141,568|2,936,372|12,319|Context
|Portfolio:children|242|2,551,326|4,130|10,542|GetValue
|Portfolio:children|121|2,547,196|2,270|21,051|GetValue/Calc
|Root:Start|270|2,515,304|2,320,146|9,315|Context
|Root:Amend MarketData|255|2,386,819|2,194,872|9,360|Context
|Root:Amend Trading|255|2,356,265|2,168,909|9,240|Context
|Root:Amend Portfolio|255|2,337,683|2,154,236|9,167|Context
|Root:Activity Portfolio|263|2,239,673|2,061,495|8,515|Context
|Root:Activity Trading|263|2,051,831|1,902,002|7,801|Context
|Root:End|278|1,892,604|1,751,597|6,807|Context
|PortfolioUpdateEvent:children|121|1,038,053|6,733|8,578|GetValue
|TradingBook|102|967,511|967,511|9,485|Db.Get
|Equity:NPV|15|462,953|398|30,863|GetValue
|Equity:NPV|15|462,554|602|30,836|GetValue/Calc
|MarketInterface:spot|15|434,324|397|28,954|GetValue
|MarketInterface:spot|15|433,926|268|28,928|GetValue/Calc
|ExternalRefData:state|15|402,714|300|26,847|GetValue
|ExternalRefData:state|15|402,413|606|26,827|GetValue/Calc
|RefData:state|15|400,969|329|26,731|GetValue
|RefData:state|15|400,640|248|26,709|GetValue/Calc
|_WorkItemEvent:book2|22,286|294,265|273,330|13|GetValue
|Clock:cutoffs|2,480|288,732|22,926|116|GetValue
|_WorkItemEvent:book1|22,286|278,650|278,650|12|GetValue
|Clock:cutoffs|20|266,022|419|13,301|GetValue/Calc
|Clock:parent|20|263,195|587|13,159|GetValue
|Clock:parent|20|262,607|508|13,130|GetValue/Calc
|TradeOpenEvent:quantity|22,286|238,604|238,604|10|GetValue
|_WorkItemEvent:item|11,143|206,393|188,894|18|GetValue
|TradeOpenEvent:premium|11,143|179,734|173,275|16|GetValue
|TradeOpenEvent:action|11,143|179,609|179,609|16|GetValue
|PortfolioUpdateEvent|12|157,345|157,345|13,112|Db.Get
|TradeOpenEvent:unitPrice|11,143|151,273|151,273|13|GetValue
|TradingBook:clock|2,200|144,096|52,984|65|GetValue
|Event:amends|11,340|134,104|134,104|11|GetValue
|RefDataUpdateEvent|9|117,933|117,933|13,103|Db.Get
|TradingBook:clock|1,100|91,112|29,624|82|GetValue/Calc
|TradingPortfolio|10|84,743|84,743|8,474|Db.Get
|Clock|5|57,230|57,230|11,446|Db.Get
|MarketInterface:source|15|29,182|337|1,945|GetValue
|MarketInterface:source|15|28,844|246|1,922|GetValue/Calc
|MarketDataSource|2|26,328|26,328|13,164|Db.Get
|Equity:refdata|15|26,041|390|1,736|GetValue
|Equity:refdata|15|25,650|284|1,710|GetValue/Calc
|ClockEvent|2|25,286|25,286|12,643|Db.Get
|MarketInterface|2|23,890|23,890|11,945|Db.Get
|ClockEvent:parent|8|23,503|342|2,937|GetValue
|Portfolio:clock|242|22,364|5,815|92|GetValue
|Portfolio:books|231|18,700|3,892|80|GetValue
|Equity|2|17,499|17,499|8,749|Db.Get
|Portfolio:clock|121|16,549|3,109|136|GetValue/Calc
|MarketDataSource:clock|30|16,105|596|536|GetValue
|Portfolio:books|121|15,581|2,041|128|GetValue/Calc
|MarketDataSource:clock|15|15,509|344|1,033|GetValue/Calc
|RootClock|1|9,798|9,798|9,798|Db.Get
|ForwardCashflow|1|6,458|6,458|6,458|Db.Get
|Entity:clock|40|2,507|998|62|GetValue
|Entity:clock|20|1,509|641|75|GetValue/Calc
|MarketInterface:sourceName|15|900|425|60|GetValue
|ForwardCashflow:NPV|11|746|280|67|GetValue
|RefDataUpdateEvent:data|58|601|601|10|GetValue
|MarketInterface:sourceName|15|475|275|31|GetValue/Calc
|RootClock:cutoffs|52|468|468|9|GetValue
|ForwardCashflow:NPV|11|465|175|42|GetValue/Calc
|Equity:assetName|15|250|250|16|GetValue
|Report:valuable|2|30|30|15|GetValue
|Report:ts1|2|16|16|8|GetValue
|Report:ts2|2|13|13|6|GetValue